In [1]:
# Modul3 5 Tarea 3 Build and Evaluate Models
## Cristian Fernandez Quiros
## Ucenfotec

In [2]:
# Step 1 Selecting and devices Data
#imports
#numpy, pandas, scipy, math, mathpolotilib
import numpy as np
import pandas as pd
import scipy
from math import sqrt
import seaborn as sns 
import matplotlib.pyplot as plt

#estimators
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import linear_model

# para normalizar
from sklearn import preprocessing

#model metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

#Metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

#cross validation
from sklearn.model_selection  import train_test_split



In [3]:
#data
rawData = pd.read_csv('default of credit card clients.csv', header=1)
rawData.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [4]:
rawData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
ID           30000 non-null int64
LIMIT_BAL    30000 non-null int64
SEX          30000 non-null int64
EDUCATION    30000 non-null int64
MARRIAGE     30000 non-null int64
AGE          30000 non-null int64
PAY_0        30000 non-null int64
PAY_2        30000 non-null int64
PAY_3        30000 non-null int64
PAY_4        30000 non-null int64
PAY_5        30000 non-null int64
PAY_6        30000 non-null int64
BILL_AMT1    30000 non-null int64
BILL_AMT2    30000 non-null int64
BILL_AMT3    30000 non-null int64
BILL_AMT4    30000 non-null int64
BILL_AMT5    30000 non-null int64
BILL_AMT6    30000 non-null int64
PAY_AMT1     30000 non-null int64
PAY_AMT2     30000 non-null int64
PAY_AMT3     30000 non-null int64
PAY_AMT4     30000 non-null int64
PAY_AMT5     30000 non-null int64
PAY_AMT6     30000 non-null int64
default      30000 non-null int64
dtypes: int64(25)
memory usage: 5.7 MB


In [5]:
rawData.columns

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'default'],
      dtype='object')

In [6]:
## 6. Crear nuevas variables agrupar

In [7]:
    def education_method(client):
        education,sex = client

        if education > 3:
            return 0
        else:
            return education

In [8]:
rawData['EDUCATION'] = rawData[['EDUCATION','SEX']].apply(education_method, axis=1)

In [9]:
bins = [18,35,60,100]
names = ["1","2","3"]
rawData["AGE"] = pd.cut(rawData["AGE"],bins,labels = names)

In [10]:
#rawData['LIMIT_BAL'] = pd.cut(rawData['LIMIT_BAL'], bins=4)

In [11]:
#1. Select the features
#features
#features = rawData.iloc[:,12:23]
features = rawData.iloc[:, [1,2,3,4,5,6,7,8,9,10,11]]
#features = rawData.iloc[:, [1,2,3,4,5]]
print('Summary of feature sample')
features.head()

Summary of feature sample


,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6
0,20000,2,2,1,1,2,2,-1,-1,-2,-2
1,120000,2,2,2,1,-1,2,0,0,0,2
2,90000,2,2,2,1,0,0,0,0,0,0
3,50000,2,2,1,2,0,0,0,0,0,0
4,50000,1,2,1,2,-1,0,-1,0,0,0


In [12]:
features.columns

Index(['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6'],
      dtype='object')

In [13]:
#2. Select the Dependent Variable

#dependent variable
depVar = rawData['default']


In [14]:
# 3.  Establish the training set for the X-variables or Feature space 
# (first 1000 rows: only for this example you will still follow a 70/30 split for your final models)
X_train = (features[: 1000])
X_train.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6
0,20000,2,2,1,1,2,2,-1,-1,-2,-2
1,120000,2,2,2,1,-1,2,0,0,0,2
2,90000,2,2,2,1,0,0,0,0,0,0
3,50000,2,2,1,2,0,0,0,0,0,0
4,50000,1,2,1,2,-1,0,-1,0,0,0


In [15]:
#4. Establish the training set for the Y-variable or dependent variable (the number of rows much match the X-training set)

#Dependent Variable Training Set (y Training)
y_train = depVar[: 1000]
y_train_count = len(y_train.index)
print('The number of observations in the Y training set are:',str(y_train_count))
y_train.head()

The number of observations in the Y training set are: 1000


0    1
1    1
2    0
3    0
4    0
Name: default, dtype: int64

In [16]:
#5. Establish the testing set for the X-Variables or Feature space

#Testing Set (X Testing)
X_test = features[-100:]
#X_test
X_test_count = len(X_test.index)
#X_test_count
print('The number of observations in the feature testing set is:',str(X_test_count))
print(X_test.head())

The number of observations in the feature testing set is: 100
       LIMIT_BAL  SEX  EDUCATION  MARRIAGE AGE  PAY_0  PAY_2  PAY_3  PAY_4  \
29900      20000    1          2         2   1      0      0     -2     -2   
29901      50000    1          2         2   1      1      2      0      0   
29902     170000    1          3         2   1      0     -1      0      0   
29903     260000    1          1         1   1     -1      0     -1     -1   
29904      60000    1          3         2   1      0      0      0      0   

       PAY_5  PAY_6  
29900     -2     -2  
29901      0      0  
29902      0      0  
29903     -1     -1  
29904      0      0  


In [17]:
#6. Establish Ground truth 
#Ground Truth (y_test) 
y_test = depVar[-100:]
#y_test
y_test_count = len(y_test.index)
#y_test_count
print('The number of observations in the Y training set are:',str(y_test_count))
y_test.head()

The number of observations in the Y training set are: 100


29900    1
29901    0
29902    0
29903    0
29904    1
Name: default, dtype: int64

In [18]:
# Cross Validation

In [19]:
#X and Y training sets:
# Now we can implement Cross Validation anytime we need to by simply running the following on the X and Y training sets:

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train)

In [20]:
# Use the shape function to double check that the split was made as needed:

X_train.shape, X_test.shape

((750, 11), (250, 11))

In [21]:
# Tuning Parameters and Model Definition
model = LinearRegression(n_jobs=10)

In [22]:
# STEP 2
# Building the Models and choosing the Right Model

In [23]:
#2. Establish variables that you will use for defining each model.
# As you can see we've established three different models with the individual variable names;
# we'll refer to each when we build the model later.

#Models
#modelSVR = SVR(gamma = 'auto')
#modelRF = RandomForestRegressor(n_estimators = 100, random_state = 123)
#modelLR = LinearRegression()
modelLDA = LinearDiscriminantAnalysis()
modelKNN = KNeighborsClassifier()
modelCART = DecisionTreeClassifier()

In [24]:
#4. View the Dependent Variable
#dependent variable
print(depVar)

0        1
1        1
2        0
3        0
4        0
        ..
29995    0
29996    0
29997    1
29998    1
29999    1
Name: default, Length: 30000, dtype: int64


In [25]:
#5. Use the model variables you established in step 2  pass the training data to them in the following format (You used the train function from caret for this when you worked in R):

model.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=10, normalize=False)

In [26]:
#6.  Linear Discriminant Analysis

modelLDA.fit(X_train,y_train)

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                           solver='svd', store_covariance=False, tol=0.0001)

In [27]:
#7. KNeighbors Classifier
cv = 5
modelKNN.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [28]:
#8. Decision Tree Classifier
cv = 5
modelCART.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [29]:
# Choosing the Right Model : by Model Fitting and Scoring

In [30]:
#1. Linear Discriminant Analysis Model Fitting and Scoring

#Model Fitting
modelLDA.fit(X_train,y_train)
print(cross_val_score(modelLDA, X_train, y_train, cv = cv))
modelLDA.score(X_train,y_train)

[0.81456954 0.78       0.81333333 0.78666667 0.81879195]


0.812

In [31]:
#2. KNeighbors Classifier Model Fitting and Scoring

cv = 5
modelKNN.fit(X_train,y_train)
print(cross_val_score(modelKNN, X_train, y_train)) 
modelKNN.score(X_train,y_train)

[0.76095618 0.784      0.73493976]


C:\Users\adria\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


0.8093333333333333

In [32]:
#3. Decision Tree Classifier Model Fitting and Scoring

cv = 5
modelCART.fit(X_train,y_train)
print(cross_val_score(modelCART, X_train, y_train, cv = cv)) 
modelCART.score(X_train,y_train)

[0.68874172 0.72       0.72       0.72666667 0.71812081]


0.9786666666666667

In [33]:
# STEP 3 Making prediction 

In [34]:
#4. Here is how it all looks together:

#Make Predictions
predictions = modelCART.predict(X_test)
predRsquared = r2_score(y_test,predictions)
rmse = sqrt(mean_squared_error(y_test, predictions))
print('R Squared: %.3f' % predRsquared)
print('RMSE: %.3f' % rmse)

R Squared: -0.400
RMSE: 0.473


In [35]:
predictions = modelKNN.predict(X_test)
y_test

print(accuracy_score(y_test, predictions))
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

0.792
[[194   6]
 [ 46   4]]
              precision    recall  f1-score   support

           0       0.81      0.97      0.88       200
           1       0.40      0.08      0.13        50

    accuracy                           0.79       250
   macro avg       0.60      0.53      0.51       250
weighted avg       0.73      0.79      0.73       250



In [36]:
predictions


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)